## 并发和并行
并发是指计算机看起来同一时刻运行多个程序。即使在单核cpu上，操作系统也会以分时间片的方式轮流执行多个程序，由于cpu速度非常，所以宏观上就是多个程序看起来同时在运行。

并行是指计算机真的是同一时刻运行多个程序。在多核cpu的情况下就可以做到微观上同一时刻运行多个程序。

并行可以显著的提高程序任务的执行速度。而并发无论分成多少个线程，但是整体的任务执行速度并没有多少提升。即使如此，并发的意义是很大的，它实现了即使在单cpu的情况下，多个程序可以同时运行，实现了对各种资源的充分里利用。在设计程序时要搞清楚并发和并行的区别，以便做出合适的选择。

In [3]:
#查看cpu内核
from multiprocessing import cpu_count
print("CPU内核数量：%s" % cpu_count()) 

CPU内核数量：8


### 多进程
在计算机操作系统中，一个进程就是一个运行着的程序，多进程技术就是如何通过运行多个程序来完成自己的任务。

### 多线程
在程序中，一个程序里面可能有多个线程，每个线程负责不同的任务，当一个线程处于阻塞状态时（比如等待一个网络数据），其它线程还可以正常运行（比如UI线程）。想并行执行线程并不容易，大部分还是并发的概念。

In [39]:
#假设我们开了地摊卖烤红薯,开几个线程同时卖红薯，观察会出现什么情况
#结果会发现大家卖的顺序可能是乱的，甚至还会出现超卖的情况
#这是因为在多线程是大家争用了共同的资源sweetpotato，造成了难以预料的问题
import threading,time
sweetpotato = 40
def sale():
    global sweetpotato
    #每个线程不断的每隔2秒卖一个红薯直到红薯卖完
    while sweetpotato>0:
        sweetpotato = sweetpotato - 1
        time.sleep(1)
        print('{}卖了一个红薯，还剩{}个红薯'.format(threading.current_thread().name, sweetpotato))
        
#创建n个卖薯哥
salethreads = [threading.Thread(target=sale,name="卖薯{}哥".format(n)) for n in range(6)] 
#一个个喊卖薯哥开始卖红薯
for salethread in salethreads:
    salethread.start()

卖薯0哥卖了一个红薯，还剩34个红薯
卖薯5哥卖了一个红薯，还剩33个红薯
卖薯2哥卖了一个红薯，还剩32个红薯
卖薯1哥卖了一个红薯，还剩31个红薯
卖薯3哥卖了一个红薯，还剩30个红薯卖薯4哥卖了一个红薯，还剩30个红薯

卖薯0哥卖了一个红薯，还剩28个红薯
卖薯5哥卖了一个红薯，还剩27个红薯
卖薯2哥卖了一个红薯，还剩26个红薯
卖薯1哥卖了一个红薯，还剩25个红薯
卖薯3哥卖了一个红薯，还剩24个红薯
卖薯4哥卖了一个红薯，还剩23个红薯
卖薯0哥卖了一个红薯，还剩22个红薯卖薯5哥卖了一个红薯，还剩22个红薯

卖薯2哥卖了一个红薯，还剩20个红薯
卖薯1哥卖了一个红薯，还剩19个红薯
卖薯3哥卖了一个红薯，还剩18个红薯
卖薯4哥卖了一个红薯，还剩17个红薯
卖薯5哥卖了一个红薯，还剩16个红薯卖薯0哥卖了一个红薯，还剩16个红薯

卖薯2哥卖了一个红薯，还剩15个红薯
卖薯1哥卖了一个红薯，还剩13个红薯
卖薯3哥卖了一个红薯，还剩12个红薯
卖薯4哥卖了一个红薯，还剩11个红薯
卖薯5哥卖了一个红薯，还剩10个红薯
卖薯2哥卖了一个红薯，还剩9个红薯卖薯0哥卖了一个红薯，还剩9个红薯

卖薯1哥卖了一个红薯，还剩7个红薯
卖薯3哥卖了一个红薯，还剩6个红薯卖薯4哥卖了一个红薯，还剩6个红薯

卖薯5哥卖了一个红薯，还剩4个红薯
卖薯2哥卖了一个红薯，还剩3个红薯卖薯0哥卖了一个红薯，还剩3个红薯

卖薯1哥卖了一个红薯，还剩1个红薯
卖薯3哥卖了一个红薯，还剩0个红薯卖薯4哥卖了一个红薯，还剩0个红薯

卖薯5哥卖了一个红薯，还剩0个红薯
卖薯2哥卖了一个红薯，还剩0个红薯卖薯0哥卖了一个红薯，还剩0个红薯

卖薯1哥卖了一个红薯，还剩0个红薯


In [40]:
#为了解决资源的访问冲突，我们通过锁机制让多个线程有条不紊的卖红薯。
#threading.RLock()提供一个
import threading,time
sweetpotato = 40
def sale(lock):
    global sweetpotato
    #每个线程不断的每隔2秒卖一个红薯直到红薯卖完
    while sweetpotato>0:
        if lock.acquire():
            if sweetpotato>0:
                sweetpotato -= 1
                time.sleep(1)
                print('{}卖了一个红薯，还剩{}个红薯'.format(threading.current_thread().name, sweetpotato))
            lock.release()
            time.sleep(1)
#创建n个卖薯哥
lock = threading.RLock()
salethreads = [threading.Thread(target=sale,args=(lock,),name="卖薯{}哥".format(n)) for n in range(6)] 
#一个个喊卖薯哥开始卖红薯
for salethread in salethreads:
    salethread.start()

卖薯0哥卖了一个红薯，还剩39个红薯
卖薯1哥卖了一个红薯，还剩38个红薯
卖薯2哥卖了一个红薯，还剩37个红薯
卖薯3哥卖了一个红薯，还剩36个红薯
卖薯4哥卖了一个红薯，还剩35个红薯
卖薯5哥卖了一个红薯，还剩34个红薯
卖薯0哥卖了一个红薯，还剩33个红薯
卖薯1哥卖了一个红薯，还剩32个红薯
卖薯2哥卖了一个红薯，还剩31个红薯
卖薯3哥卖了一个红薯，还剩30个红薯
卖薯4哥卖了一个红薯，还剩29个红薯
卖薯5哥卖了一个红薯，还剩28个红薯
卖薯0哥卖了一个红薯，还剩27个红薯
卖薯1哥卖了一个红薯，还剩26个红薯
卖薯2哥卖了一个红薯，还剩25个红薯
卖薯3哥卖了一个红薯，还剩24个红薯
卖薯4哥卖了一个红薯，还剩23个红薯
卖薯5哥卖了一个红薯，还剩22个红薯
卖薯0哥卖了一个红薯，还剩21个红薯
卖薯1哥卖了一个红薯，还剩20个红薯
卖薯2哥卖了一个红薯，还剩19个红薯
卖薯3哥卖了一个红薯，还剩18个红薯
卖薯4哥卖了一个红薯，还剩17个红薯
卖薯5哥卖了一个红薯，还剩16个红薯
卖薯0哥卖了一个红薯，还剩15个红薯
卖薯1哥卖了一个红薯，还剩14个红薯
卖薯2哥卖了一个红薯，还剩13个红薯
卖薯3哥卖了一个红薯，还剩12个红薯
卖薯4哥卖了一个红薯，还剩11个红薯
卖薯5哥卖了一个红薯，还剩10个红薯
卖薯0哥卖了一个红薯，还剩9个红薯
卖薯1哥卖了一个红薯，还剩8个红薯
卖薯2哥卖了一个红薯，还剩7个红薯
卖薯3哥卖了一个红薯，还剩6个红薯
卖薯4哥卖了一个红薯，还剩5个红薯
卖薯5哥卖了一个红薯，还剩4个红薯
卖薯0哥卖了一个红薯，还剩3个红薯
卖薯1哥卖了一个红薯，还剩2个红薯
卖薯2哥卖了一个红薯，还剩1个红薯
卖薯3哥卖了一个红薯，还剩0个红薯


In [ ]:
#随着红薯规模的扩大，有人烤红薯有人卖红薯
#所以我们要定义两个线程，一个烤红薯一个卖红薯
#然后招聘10个员工，编号奇数烤红薯，编号偶数卖红薯
#烤红薯的人每次花5秒钟烤5个红薯，卖红薯的人一秒钟卖1个红薯
import threading,time
roasted_sweetpatato = 0

def roast(lock):
    global roasted_sweetpatato
    if lock.acquire():
        time.sleep(5)
        roasted_sweetpatato += 5
        print("{}烤了5个红薯，现在一共是{}个".format(threading.current_thread.name,roasted_sweetpatato))
        lock.release()

def sale(lock):
    global roasted_sweetpatato
    if roasted_sweetpatato>0 and lock.acquire():
        time.sleep(1)
        if roasted_sweetpatato>0:
            roasted_sweetpatato -= 1
            print("{}卖了1个红薯，现在一共是{}个".format(threading.current_thread.name,roasted_sweetpatato))
        lock.release()
        
lock = threading.Lock()

threads = []
for index in range(4):
    threads.append(threading.Thread(target=(sale if index%2 else roast),
                               args=(lock,),
                               name=("烤薯{}姐" if index%2 else "卖薯{}哥").format(index)))
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
